In [10]:
# import urllib.request
# api_key = ''
# crypto = 'DOGE'
# interval = '1d'
# # url = "https://api.nomics.com/v1/currencies/ticker?key=" + api_key + \
# #       "&ids=" + crypto + "&interval=" + interval + "&convert=USD&per-page=100&page=1"

# start_date = '2020-01-01'
# end_date = '2020-12-31'
# url = "https://api.nomics.com/v1/currencies/sparkline?key=" + api_key + \
#       "&ids=" + crypto + "&start=" + start_date + "T00%3A00%3A00Z&end=" + end_date + "T00%3A00%3A00Z" + "&interval=" + interval

# print(urllib.request.urlopen(url).read())

b'[{"currency":"DOGE","timestamps":["2020-01-06T00:00:00Z","2020-01-13T00:00:00Z","2020-01-20T00:00:00Z","2020-01-27T00:00:00Z","2020-02-03T00:00:00Z","2020-02-10T00:00:00Z","2020-02-17T00:00:00Z","2020-02-24T00:00:00Z","2020-03-02T00:00:00Z","2020-03-09T00:00:00Z","2020-03-16T00:00:00Z","2020-03-23T00:00:00Z","2020-03-30T00:00:00Z","2020-04-06T00:00:00Z","2020-04-13T00:00:00Z","2020-04-20T00:00:00Z","2020-04-27T00:00:00Z","2020-05-04T00:00:00Z","2020-05-11T00:00:00Z","2020-05-18T00:00:00Z","2020-05-25T00:00:00Z","2020-06-01T00:00:00Z","2020-06-08T00:00:00Z","2020-06-15T00:00:00Z","2020-06-22T00:00:00Z","2020-06-29T00:00:00Z","2020-07-06T00:00:00Z","2020-07-13T00:00:00Z","2020-07-20T00:00:00Z","2020-07-27T00:00:00Z","2020-08-03T00:00:00Z","2020-08-10T00:00:00Z","2020-08-17T00:00:00Z","2020-08-24T00:00:00Z","2020-08-31T00:00:00Z","2020-09-07T00:00:00Z","2020-09-14T00:00:00Z","2020-09-21T00:00:00Z","2020-09-28T00:00:00Z","2020-10-05T00:00:00Z","2020-10-12T00:00:00Z","2020-10-19T00:00:00Z

In [11]:
# Description: This program attempts to predict the future price of Dogecoin

In [12]:
# Import the libraries
import numpy as np
import pandas as pd

In [18]:
# Load the data
df = pd.read_csv('doge.csv')

# Set date as the index 
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)

# Remove unwanted data
df = df[['PriceUSD']]

# Show the data
df.head()

,PriceUSD
date,
2014-01-23,0.001541
2014-01-24,0.001888
2014-01-25,0.001937
2014-01-26,0.001937
2014-01-27,0.001618


In [19]:
# Create a variable to store the number of days into the future that we want to predict
prediction_days = 1

# Create a new column called Prediction
df['Prediction'] = df['PriceUSD'].shift(-prediction_days)

# Show the data
df

,PriceUSD,Prediction
date,,
2014-01-23,0.001541,0.001888
2014-01-24,0.001888,0.001937
2014-01-25,0.001937,0.001937
2014-01-26,0.001937,0.001618
2014-01-27,0.001618,0.001651
...,...,...
2021-06-22,0.188829,0.234789
2021-06-23,0.234789,0.263518
2021-06-24,0.263518,0.240247
